In [5]:
import pandas as pd
import numpy as np

PATH = '/home/kai/data/kaggle/homecredit/'

train = pd.read_csv(PATH + 'application_train.csv')
test = pd.read_csv(PATH + 'application_test.csv')
print('done')

done


In [6]:
bureau = pd.read_csv(PATH + 'bureau.csv')
print('bureau')
bureau_balance = pd.read_csv(PATH + 'bureau_balance.csv')
print('bureau_balance')
installments_payments = pd.read_csv(PATH + 'installments_payments.csv')
print('installments_payments done')
credit_card_balance = pd.read_csv(PATH + 'credit_card_balance.csv')
print('credit_card_balance')
POS_CASH_balance = pd.read_csv(PATH + 'POS_CASH_balance.csv')
print('POS_CASH_balance')
credit_card_balance = pd.read_csv(PATH + 'credit_card_balance.csv')
print('credit_card_balance')
previous_application = pd.read_csv(PATH + 'previous_application.csv')
print('previous_application')

bureau
bureau_balance
installments_payments done
credit_card_balance
POS_CASH_balance
credit_card_balance
previous_application


# Functions

In [7]:
import pandas as pd
import os
import gc
import numpy as np

def ratio_name(numerator, denominator): return numerator + '_divide_' + denominator

def ratio(df, numerator, denominator):
    return df[numerator] / df[denominator]

def substraction_name(col1, col2): return col1 + '_minus_' + col2

def substraction(df, col1, col2):
    return df[col1] - df[col2]

In [8]:
def numerical_agg(df, gp_col, agg_col, df_name, extrafunc_list = None):
    
    agg_list = ['max', 'min', 'std','mean']
    '''
    Possible extrafunc_list: ['sum','median','two_minus_one_third','positive_count', 'negative_count','standard_error', 'trimmed_mean_10_pct', 'trimmed_mean_25_pct','normed_std', 'max_minus_min','one_third','two_third']
    '''
    if extrafunc_list:
        if 'one_third' in extrafunc_list:
            def one_third(series):
                return series.quantile(1/3)
            
        if 'two_third' in extrafunc_list:
            def two_third(series):
                return series.quantile(2/3)
            
        if 'max_minus_min' in extrafunc_list:
            def max_minus_min(series):
                return (series.max() - series.min())
        
        if 'two_minus_one_third' in extrafunc_list:
#             print('two_minus_one')
#             print('get two minus one')
            def two_minus_one_third(series):
                return(series.quantile(0.66666) - series.quantile(0.33333))
       
        if 'positive_count' in extrafunc_list:
#             print('get positive count')
            def positive_count(series):
                return pd.Series(series > 0).sum()

        if 'negative_count' in extrafunc_list:
#             print('get negative count')
            def negative_count(series):
                return pd.Series(series < 0).sum()

        if 'standard_error' in extrafunc_list:
#             print('get std-error')
            def standard_error(series):
                return series.std()/np.sqrt(len(series))

        if 'normed_std' in extrafunc_list:
#             print('get normed_std')
            def normed_std(series):
                return series.std()/series.mean()

        if 'trimmed_mean_10_pct' in extrafunc_list:
#             print('get_trimmed 10 pct')
            def trimmed_mean_10_pct(series):
                return stats.trim_mean(series.dropna(), 0.1)

        if 'trimmed_mean_25_pct' in extrafunc_list:
#             print('get_trimmed 25 pct')
            def trimmed_mean_25_pct(series):
                return stats.trim_mean(series.dropna(), 0.25)
        
        list_tocall = []
        for i in extrafunc_list:
            if i not in set(['sum', 'median']):
                list_tocall.append(eval(i))
            
    if extrafunc_list != None:   
        agg_list.extend(list_tocall)
    
    _df = df.groupby(gp_col).agg({agg_col:agg_list})
    columns = []
    for pre in _df.columns.levels[0]:
        for middle in _df.columns.levels[1]:
            columns.append('%s_%s_%s' %(df_name,pre,middle))
    _df.columns = columns
    
    return _df.reset_index()

# Install

In [9]:
install = installments_payments.copy()

In [10]:
install[substraction_name('DAYS_INSTALMENT', 'DAYS_ENTRY_PAYMENT')] =\
substraction(install, 'DAYS_INSTALMENT' ,'DAYS_ENTRY_PAYMENT')

install[substraction_name('AMT_PAYMENT', 'AMT_INSTALMENT')] = \
substraction(install, 'AMT_PAYMENT' ,'AMT_INSTALMENT')

install[ratio_name('AMT_PAYMENT', 'AMT_INSTALMENT')] = \
ratio(install, 'AMT_PAYMENT' ,'AMT_INSTALMENT')

install['late_days'] = 0
x = install['DAYS_INSTALMENT_minus_DAYS_ENTRY_PAYMENT'] < 0
install['late_days'][x] = install['DAYS_INSTALMENT_minus_DAYS_ENTRY_PAYMENT'][x]

/home/kai/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


# Install Score

In [11]:
install['score'] = (install['AMT_PAYMENT_divide_AMT_INSTALMENT']*(-20) + 21)\
* np.exp((install['late_days'])**3/100)* (1/(1 + abs(install['DAYS_ENTRY_PAYMENT']/30) ))

In [12]:
install.columns.values

array(['SK_ID_PREV', 'SK_ID_CURR', 'NUM_INSTALMENT_VERSION',
       'NUM_INSTALMENT_NUMBER', 'DAYS_INSTALMENT', 'DAYS_ENTRY_PAYMENT',
       'AMT_INSTALMENT', 'AMT_PAYMENT',
       'DAYS_INSTALMENT_minus_DAYS_ENTRY_PAYMENT',
       'AMT_PAYMENT_minus_AMT_INSTALMENT',
       'AMT_PAYMENT_divide_AMT_INSTALMENT', 'late_days', 'score'], dtype=object)

In [13]:
gp_col = 'SK_ID_CURR'
agg_col = ['DAYS_INSTALMENT_minus_DAYS_ENTRY_PAYMENT',
          'AMT_PAYMENT_minus_AMT_INSTALMENT',
           'AMT_PAYMENT_divide_AMT_INSTALMENT',
          'AMT_INSTALMENT',
          'DAYS_ENTRY_PAYMENT',
          'AMT_PAYMENT',
          'late_days','DAYS_INSTALMENT','score']

'''
    Possible extrafunc_list: ['sum','median','two_minus_one_third','positive_count', 'negative_count','standard_error', 'trimmed_mean_10_pct', 'trimmed_mean_25_pct','normed_std', 'max_minus_min','one_third','two_third']
    '''
extrafunc_list = [['negative_count'],['negative_count'],None,None,None,None,None,None,None]
for i in range(len(extrafunc_list)):
    print(agg_col[i])
    train = train.merge(numerical_agg(install, gp_col, agg_col[i], 'installments',extrafunc_list = extrafunc_list[i]),\
                       on = 'SK_ID_CURR', how='left')
    test = test.merge(numerical_agg(install, gp_col, agg_col[i], 'installments',extrafunc_list = extrafunc_list[i]),\
                       on = 'SK_ID_CURR', how='left')

DAYS_INSTALMENT_minus_DAYS_ENTRY_PAYMENT
AMT_PAYMENT_minus_AMT_INSTALMENT
AMT_PAYMENT_divide_AMT_INSTALMENT
AMT_INSTALMENT
DAYS_ENTRY_PAYMENT
AMT_PAYMENT
late_days
DAYS_INSTALMENT
score


In [14]:
print(len(test.columns.values), train.columns.values)

159 ['SK_ID_CURR' 'TARGET' 'NAME_CONTRACT_TYPE' 'CODE_GENDER' 'FLAG_OWN_CAR'
 'FLAG_OWN_REALTY' 'CNT_CHILDREN' 'AMT_INCOME_TOTAL' 'AMT_CREDIT'
 'AMT_ANNUITY' 'AMT_GOODS_PRICE' 'NAME_TYPE_SUITE' 'NAME_INCOME_TYPE'
 'NAME_EDUCATION_TYPE' 'NAME_FAMILY_STATUS' 'NAME_HOUSING_TYPE'
 'REGION_POPULATION_RELATIVE' 'DAYS_BIRTH' 'DAYS_EMPLOYED'
 'DAYS_REGISTRATION' 'DAYS_ID_PUBLISH' 'OWN_CAR_AGE' 'FLAG_MOBIL'
 'FLAG_EMP_PHONE' 'FLAG_WORK_PHONE' 'FLAG_CONT_MOBILE' 'FLAG_PHONE'
 'FLAG_EMAIL' 'OCCUPATION_TYPE' 'CNT_FAM_MEMBERS' 'REGION_RATING_CLIENT'
 'REGION_RATING_CLIENT_W_CITY' 'WEEKDAY_APPR_PROCESS_START'
 'HOUR_APPR_PROCESS_START' 'REG_REGION_NOT_LIVE_REGION'
 'REG_REGION_NOT_WORK_REGION' 'LIVE_REGION_NOT_WORK_REGION'
 'REG_CITY_NOT_LIVE_CITY' 'REG_CITY_NOT_WORK_CITY'
 'LIVE_CITY_NOT_WORK_CITY' 'ORGANIZATION_TYPE' 'EXT_SOURCE_1'
 'EXT_SOURCE_2' 'EXT_SOURCE_3' 'APARTMENTS_AVG' 'BASEMENTAREA_AVG'
 'YEARS_BEGINEXPLUATATION_AVG' 'YEARS_BUILD_AVG' 'COMMONAREA_AVG'
 'ELEVATORS_AVG' 'ENTRANCES_AVG' 'F

# Save Install

In [15]:
print(train.shape, test.shape,train.columns.values)
# PATH = '/home/kai/data/kaggle/homecredit/'
# train.to_pickle(PATH + 'train_with_install.pkl')
# test.to_pickle(PATH + 'test_with_install.pkl')

(307511, 160) (48744, 159) ['SK_ID_CURR' 'TARGET' 'NAME_CONTRACT_TYPE' 'CODE_GENDER' 'FLAG_OWN_CAR'
 'FLAG_OWN_REALTY' 'CNT_CHILDREN' 'AMT_INCOME_TOTAL' 'AMT_CREDIT'
 'AMT_ANNUITY' 'AMT_GOODS_PRICE' 'NAME_TYPE_SUITE' 'NAME_INCOME_TYPE'
 'NAME_EDUCATION_TYPE' 'NAME_FAMILY_STATUS' 'NAME_HOUSING_TYPE'
 'REGION_POPULATION_RELATIVE' 'DAYS_BIRTH' 'DAYS_EMPLOYED'
 'DAYS_REGISTRATION' 'DAYS_ID_PUBLISH' 'OWN_CAR_AGE' 'FLAG_MOBIL'
 'FLAG_EMP_PHONE' 'FLAG_WORK_PHONE' 'FLAG_CONT_MOBILE' 'FLAG_PHONE'
 'FLAG_EMAIL' 'OCCUPATION_TYPE' 'CNT_FAM_MEMBERS' 'REGION_RATING_CLIENT'
 'REGION_RATING_CLIENT_W_CITY' 'WEEKDAY_APPR_PROCESS_START'
 'HOUR_APPR_PROCESS_START' 'REG_REGION_NOT_LIVE_REGION'
 'REG_REGION_NOT_WORK_REGION' 'LIVE_REGION_NOT_WORK_REGION'
 'REG_CITY_NOT_LIVE_CITY' 'REG_CITY_NOT_WORK_CITY'
 'LIVE_CITY_NOT_WORK_CITY' 'ORGANIZATION_TYPE' 'EXT_SOURCE_1'
 'EXT_SOURCE_2' 'EXT_SOURCE_3' 'APARTMENTS_AVG' 'BASEMENTAREA_AVG'
 'YEARS_BEGINEXPLUATATION_AVG' 'YEARS_BUILD_AVG' 'COMMONAREA_AVG'
 'ELEVATORS_

# Previous

In [16]:
PATH = '/home/kai/data/kaggle/homecredit/'

# train = pd.read_pickle(PATH + 'train_cleaned.pkl')
# test = pd.read_pickle(PATH + 'test_cleaned.pkl')
prev = previous_application.copy()
print('done')

done


In [17]:
# Ratio
prev[ratio_name('AMT_APPLICATION', 'AMT_CREDIT')] = ratio(prev, 'AMT_APPLICATION', 'AMT_CREDIT')

prev[ratio_name('AMT_DOWN_PAYMENT', 'RATE_DOWN_PAYMENT')] = ratio(prev, 'AMT_DOWN_PAYMENT', 'RATE_DOWN_PAYMENT')

prev[ratio_name('AMT_CREDIT', 'CNT_PAYMENT')] = ratio(prev, 'AMT_CREDIT', 'CNT_PAYMENT')

# Sub
prev[substraction_name('DAYS_LAST_DUE', 'DAYS_FIRST_DUE')] = substraction(prev, 'DAYS_LAST_DUE', 'DAYS_FIRST_DUE')

prev[substraction_name('AMT_APPLICATION', 'AMT_CREDIT')] = substraction(prev, 'AMT_APPLICATION', 'AMT_CREDIT')


In [18]:
print(prev.columns)

Index(['SK_ID_PREV', 'SK_ID_CURR', 'NAME_CONTRACT_TYPE', 'AMT_ANNUITY',
       'AMT_APPLICATION', 'AMT_CREDIT', 'AMT_DOWN_PAYMENT', 'AMT_GOODS_PRICE',
       'WEEKDAY_APPR_PROCESS_START', 'HOUR_APPR_PROCESS_START',
       'FLAG_LAST_APPL_PER_CONTRACT', 'NFLAG_LAST_APPL_IN_DAY',
       'RATE_DOWN_PAYMENT', 'RATE_INTEREST_PRIMARY',
       'RATE_INTEREST_PRIVILEGED', 'NAME_CASH_LOAN_PURPOSE',
       'NAME_CONTRACT_STATUS', 'DAYS_DECISION', 'NAME_PAYMENT_TYPE',
       'CODE_REJECT_REASON', 'NAME_TYPE_SUITE', 'NAME_CLIENT_TYPE',
       'NAME_GOODS_CATEGORY', 'NAME_PORTFOLIO', 'NAME_PRODUCT_TYPE',
       'CHANNEL_TYPE', 'SELLERPLACE_AREA', 'NAME_SELLER_INDUSTRY',
       'CNT_PAYMENT', 'NAME_YIELD_GROUP', 'PRODUCT_COMBINATION',
       'DAYS_FIRST_DRAWING', 'DAYS_FIRST_DUE', 'DAYS_LAST_DUE_1ST_VERSION',
       'DAYS_LAST_DUE', 'DAYS_TERMINATION', 'NFLAG_INSURED_ON_APPROVAL',
       'AMT_APPLICATION_divide_AMT_CREDIT',
       'AMT_DOWN_PAYMENT_divide_RATE_DOWN_PAYMENT',
       'AMT_CREDIT_divid

In [19]:
# Drop Columns for numerical

# For now drop: SELLERPLACE_AREA, RATE_INTEREST_PRIMARY, RATE_INTEREST_PRIVILEGED
numerical_cols = ['AMT_APPLICATION_divide_AMT_CREDIT', 'AMT_ANNUITY', 'AMT_APPLICATION', 'AMT_CREDIT', 'AMT_DOWN_PAYMENT', 'AMT_GOODS_PRICE', 'HOUR_APPR_PROCESS_START', 'RATE_DOWN_PAYMENT',  'DAYS_DECISION', 'CNT_PAYMENT', 'DAYS_FIRST_DRAWING', 'DAYS_FIRST_DUE', 'DAYS_LAST_DUE_1ST_VERSION', 'DAYS_LAST_DUE', 'DAYS_TERMINATION',
       'AMT_DOWN_PAYMENT_divide_RATE_DOWN_PAYMENT',
       'AMT_CREDIT_divide_CNT_PAYMENT', 'DAYS_LAST_DUE_minus_DAYS_FIRST_DUE',
       'AMT_APPLICATION_minus_AMT_CREDIT']

extrafunc_list = [_ for _ in range(len(numerical_cols))]
for i in range(len(numerical_cols)):
    if i == 0:
        extrafunc_list[i] = ['positive_count']
    else:
        extrafunc_list[i] = None

In [20]:
gp_col = 'SK_ID_CURR'
for i in range(len(extrafunc_list)):
    print(numerical_cols[i])
    train = train.merge(numerical_agg(prev, gp_col, numerical_cols[i], 'prev', extrafunc_list = extrafunc_list[i]), on = 'SK_ID_CURR', how='left')
    test = test.merge(numerical_agg(prev, gp_col, numerical_cols[i], 'prev', extrafunc_list = extrafunc_list[i]), on = 'SK_ID_CURR', how='left')

AMT_APPLICATION_divide_AMT_CREDIT
AMT_ANNUITY
AMT_APPLICATION
AMT_CREDIT
AMT_DOWN_PAYMENT
AMT_GOODS_PRICE
HOUR_APPR_PROCESS_START
RATE_DOWN_PAYMENT
DAYS_DECISION
CNT_PAYMENT
DAYS_FIRST_DRAWING
DAYS_FIRST_DUE
DAYS_LAST_DUE_1ST_VERSION
DAYS_LAST_DUE
DAYS_TERMINATION
AMT_DOWN_PAYMENT_divide_RATE_DOWN_PAYMENT
AMT_CREDIT_divide_CNT_PAYMENT
DAYS_LAST_DUE_minus_DAYS_FIRST_DUE
AMT_APPLICATION_minus_AMT_CREDIT


In [21]:
categorical_cols = [col for col in prev.columns if prev[col].dtype == 'object']
categorical_cols.append('NFLAG_LAST_APPL_IN_DAY')
categorical_cols.append('NFLAG_INSURED_ON_APPROVAL')
categorical_cols

['NAME_CONTRACT_TYPE',
 'WEEKDAY_APPR_PROCESS_START',
 'FLAG_LAST_APPL_PER_CONTRACT',
 'NAME_CASH_LOAN_PURPOSE',
 'NAME_CONTRACT_STATUS',
 'NAME_PAYMENT_TYPE',
 'CODE_REJECT_REASON',
 'NAME_TYPE_SUITE',
 'NAME_CLIENT_TYPE',
 'NAME_GOODS_CATEGORY',
 'NAME_PORTFOLIO',
 'NAME_PRODUCT_TYPE',
 'CHANNEL_TYPE',
 'NAME_SELLER_INDUSTRY',
 'NAME_YIELD_GROUP',
 'PRODUCT_COMBINATION',
 'NFLAG_LAST_APPL_IN_DAY',
 'NFLAG_INSURED_ON_APPROVAL']

In [22]:
def _set_type(series, dtype):
    _max, _min = max(series), min(series)
    if dtype == 'uint':
        if _max <= 255: return np.uint8
        elif _max <= 65535: return np.uint16
        elif _max <= 4294967295: return np.uint32
        else: return np.uint64
    elif dtype == 'int':
        if _min >= -128 and _max <= 127: return np.int8
        elif _min >=-32768 and _max <= 32767: return np.int16
        elif _min >= -2147483648 and _max <= 2147483647: return np.int32
        else: return np.int64
    elif dtype == 'float':
        if max(abs(_min), _max) <= 3.4028235e+38: return np.float32
        else: return np.float64

def split_categorical_feature(df, group_col, calc_col):
    tmp_df = pd.concat([df[group_col], pd.get_dummies(df[calc_col], prefix=calc_col)], axis=1).groupby(by=group_col).sum().reset_index()
    dtype = {x: _set_type(tmp_df[x], 'uint') for x in tmp_df.columns if x != group_col}
    for x in tmp_df:
        if x in df.columns:
            dtype[x] = df[x].dtype
    return tmp_df.astype(dtype)

In [23]:
for col in categorical_cols:
    print(col)
    train = train.merge(split_categorical_feature(prev, 'SK_ID_CURR', col), on='SK_ID_CURR', how='left')
    test = test.merge(split_categorical_feature(prev, 'SK_ID_CURR', col), on='SK_ID_CURR', how='left')
    gc.collect()

NAME_CONTRACT_TYPE
WEEKDAY_APPR_PROCESS_START
FLAG_LAST_APPL_PER_CONTRACT
NAME_CASH_LOAN_PURPOSE
NAME_CONTRACT_STATUS
NAME_PAYMENT_TYPE
CODE_REJECT_REASON
NAME_TYPE_SUITE
NAME_CLIENT_TYPE
NAME_GOODS_CATEGORY
NAME_PORTFOLIO
NAME_PRODUCT_TYPE
CHANNEL_TYPE
NAME_SELLER_INDUSTRY
NAME_YIELD_GROUP
PRODUCT_COMBINATION
NFLAG_LAST_APPL_IN_DAY
NFLAG_INSURED_ON_APPROVAL


# Save Prev

In [24]:
print(train.shape, test.shape)
# PATH = '/home/kai/data/kaggle/homecredit/'
# train.to_pickle(PATH + 'train_with_prev.pkl')
# test.to_pickle(PATH + 'test_with_prev.pkl')

(307511, 384) (48744, 383)


# Credit Card

In [25]:
# PATH = '/home/kai/data/kaggle/homecredit/'
# train = pd.read_csv(PATH + 'application_train.csv')
# print('train')
# test = pd.read_csv(PATH + 'application_test.csv')
# print('test')
credit = credit_card_balance.copy()
print('done')

done


In [26]:
# Substraction
credit[substraction_name('AMT_CREDIT_LIMIT_ACTUAL', 'AMT_BALANCE')] = \
substraction(credit, 'AMT_CREDIT_LIMIT_ACTUAL', 'AMT_BALANCE')

# Ratio
top = ['AMT_DRAWINGS_ATM_CURRENT', 'AMT_DRAWINGS_POS_CURRENT', 'AMT_DRAWINGS_OTHER_CURRENT',\
      'CNT_DRAWINGS_ATM_CURRENT', 'CNT_DRAWINGS_OTHER_CURRENT', 'CNT_DRAWINGS_POS_CURRENT',\
      'AMT_DRAWINGS_ATM_CURRENT', 'AMT_DRAWINGS_POS_CURRENT', 'AMT_DRAWINGS_OTHER_CURRENT', 'AMT_DRAWINGS_CURRENT']

bot = ['AMT_DRAWINGS_CURRENT', 'AMT_DRAWINGS_CURRENT','AMT_DRAWINGS_CURRENT', \
       'CNT_DRAWINGS_CURRENT', 'CNT_DRAWINGS_CURRENT', 'CNT_DRAWINGS_CURRENT',\
       'CNT_DRAWINGS_ATM_CURRENT', 'CNT_DRAWINGS_POS_CURRENT', 'CNT_DRAWINGS_OTHER_CURRENT', 'CNT_DRAWINGS_CURRENT']
for i in range(len(top)):
    print(top[i],bot[i])
    credit[ratio_name(top[i], bot[i])] = \
    ratio(credit, top[i], bot[i])

AMT_DRAWINGS_ATM_CURRENT AMT_DRAWINGS_CURRENT
AMT_DRAWINGS_POS_CURRENT AMT_DRAWINGS_CURRENT
AMT_DRAWINGS_OTHER_CURRENT AMT_DRAWINGS_CURRENT
CNT_DRAWINGS_ATM_CURRENT CNT_DRAWINGS_CURRENT
CNT_DRAWINGS_OTHER_CURRENT CNT_DRAWINGS_CURRENT
CNT_DRAWINGS_POS_CURRENT CNT_DRAWINGS_CURRENT
AMT_DRAWINGS_ATM_CURRENT CNT_DRAWINGS_ATM_CURRENT
AMT_DRAWINGS_POS_CURRENT CNT_DRAWINGS_POS_CURRENT
AMT_DRAWINGS_OTHER_CURRENT CNT_DRAWINGS_OTHER_CURRENT
AMT_DRAWINGS_CURRENT CNT_DRAWINGS_CURRENT


In [27]:
credit['score_DPD'] = 0
credit['score_DPD'] = (credit['SK_DPD']**3 + 5* credit['SK_DPD_DEF']**3 + 1) * (1/(1 + abs(credit['MONTHS_BALANCE']) ))

In [28]:
col = [i for i in list(credit.columns.values) if credit[i].dtype!= 'object']
print(col)

['SK_ID_PREV', 'SK_ID_CURR', 'MONTHS_BALANCE', 'AMT_BALANCE', 'AMT_CREDIT_LIMIT_ACTUAL', 'AMT_DRAWINGS_ATM_CURRENT', 'AMT_DRAWINGS_CURRENT', 'AMT_DRAWINGS_OTHER_CURRENT', 'AMT_DRAWINGS_POS_CURRENT', 'AMT_INST_MIN_REGULARITY', 'AMT_PAYMENT_CURRENT', 'AMT_PAYMENT_TOTAL_CURRENT', 'AMT_RECEIVABLE_PRINCIPAL', 'AMT_RECIVABLE', 'AMT_TOTAL_RECEIVABLE', 'CNT_DRAWINGS_ATM_CURRENT', 'CNT_DRAWINGS_CURRENT', 'CNT_DRAWINGS_OTHER_CURRENT', 'CNT_DRAWINGS_POS_CURRENT', 'CNT_INSTALMENT_MATURE_CUM', 'SK_DPD', 'SK_DPD_DEF', 'AMT_CREDIT_LIMIT_ACTUAL_minus_AMT_BALANCE', 'AMT_DRAWINGS_ATM_CURRENT_divide_AMT_DRAWINGS_CURRENT', 'AMT_DRAWINGS_POS_CURRENT_divide_AMT_DRAWINGS_CURRENT', 'AMT_DRAWINGS_OTHER_CURRENT_divide_AMT_DRAWINGS_CURRENT', 'CNT_DRAWINGS_ATM_CURRENT_divide_CNT_DRAWINGS_CURRENT', 'CNT_DRAWINGS_OTHER_CURRENT_divide_CNT_DRAWINGS_CURRENT', 'CNT_DRAWINGS_POS_CURRENT_divide_CNT_DRAWINGS_CURRENT', 'AMT_DRAWINGS_ATM_CURRENT_divide_CNT_DRAWINGS_ATM_CURRENT', 'AMT_DRAWINGS_POS_CURRENT_divide_CNT_DRAWINGS

# Numerical Cols

In [29]:
numerical_cols = ['SK_DPD', 'SK_DPD_DEF', 'AMT_BALANCE', 'AMT_CREDIT_LIMIT_ACTUAL', 'AMT_DRAWINGS_ATM_CURRENT', 'AMT_DRAWINGS_CURRENT', 'AMT_DRAWINGS_OTHER_CURRENT', 'AMT_DRAWINGS_POS_CURRENT', 'AMT_INST_MIN_REGULARITY', 'AMT_PAYMENT_CURRENT', 'AMT_PAYMENT_TOTAL_CURRENT', 'AMT_RECEIVABLE_PRINCIPAL', 'AMT_RECIVABLE', 'AMT_TOTAL_RECEIVABLE', 'CNT_DRAWINGS_ATM_CURRENT', 'CNT_DRAWINGS_CURRENT', 'CNT_DRAWINGS_OTHER_CURRENT', 'CNT_DRAWINGS_POS_CURRENT', 'CNT_INSTALMENT_MATURE_CUM', 'AMT_CREDIT_LIMIT_ACTUAL_minus_AMT_BALANCE', 'AMT_DRAWINGS_ATM_CURRENT_divide_AMT_DRAWINGS_CURRENT', 'AMT_DRAWINGS_POS_CURRENT_divide_AMT_DRAWINGS_CURRENT', 'AMT_DRAWINGS_OTHER_CURRENT_divide_AMT_DRAWINGS_CURRENT', 'CNT_DRAWINGS_ATM_CURRENT_divide_CNT_DRAWINGS_CURRENT', 'CNT_DRAWINGS_OTHER_CURRENT_divide_CNT_DRAWINGS_CURRENT', 'CNT_DRAWINGS_POS_CURRENT_divide_CNT_DRAWINGS_CURRENT', 'AMT_DRAWINGS_ATM_CURRENT_divide_CNT_DRAWINGS_ATM_CURRENT', 'AMT_DRAWINGS_POS_CURRENT_divide_CNT_DRAWINGS_POS_CURRENT', 'AMT_DRAWINGS_OTHER_CURRENT_divide_CNT_DRAWINGS_OTHER_CURRENT', 'AMT_DRAWINGS_CURRENT_divide_CNT_DRAWINGS_CURRENT', 'score_DPD']

In [30]:
numerical_len = len(numerical_cols)

extrafunc_list = [_ for _ in range(len(numerical_cols))]

for i in range(len(numerical_cols)):
    if i == 0 or i == 1:
        extrafunc_list[i] = ['positive_count']
    else:
        if i == 2:
            extrafunc_list[i] = ['positive_count', 'negative_count']
        else:
            extrafunc_list[i] = None
print(extrafunc_list)

[['positive_count'], ['positive_count'], ['positive_count', 'negative_count'], None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]


In [31]:
gp_col = 'SK_ID_CURR'
for i in range(len(extrafunc_list)):
    print(numerical_cols[i])
    test = test.merge(numerical_agg(credit, gp_col, numerical_cols[i], 'credit_card', extrafunc_list = extrafunc_list[i]), on = 'SK_ID_CURR', how='left')
    train = train.merge(numerical_agg(credit, gp_col, numerical_cols[i], 'credit_card', extrafunc_list = extrafunc_list[i]), on = 'SK_ID_CURR', how='left')

SK_DPD
SK_DPD_DEF
AMT_BALANCE
AMT_CREDIT_LIMIT_ACTUAL
AMT_DRAWINGS_ATM_CURRENT
AMT_DRAWINGS_CURRENT
AMT_DRAWINGS_OTHER_CURRENT
AMT_DRAWINGS_POS_CURRENT
AMT_INST_MIN_REGULARITY
AMT_PAYMENT_CURRENT
AMT_PAYMENT_TOTAL_CURRENT
AMT_RECEIVABLE_PRINCIPAL
AMT_RECIVABLE
AMT_TOTAL_RECEIVABLE
CNT_DRAWINGS_ATM_CURRENT
CNT_DRAWINGS_CURRENT
CNT_DRAWINGS_OTHER_CURRENT
CNT_DRAWINGS_POS_CURRENT
CNT_INSTALMENT_MATURE_CUM
AMT_CREDIT_LIMIT_ACTUAL_minus_AMT_BALANCE
AMT_DRAWINGS_ATM_CURRENT_divide_AMT_DRAWINGS_CURRENT
AMT_DRAWINGS_POS_CURRENT_divide_AMT_DRAWINGS_CURRENT
AMT_DRAWINGS_OTHER_CURRENT_divide_AMT_DRAWINGS_CURRENT
CNT_DRAWINGS_ATM_CURRENT_divide_CNT_DRAWINGS_CURRENT
CNT_DRAWINGS_OTHER_CURRENT_divide_CNT_DRAWINGS_CURRENT
CNT_DRAWINGS_POS_CURRENT_divide_CNT_DRAWINGS_CURRENT
AMT_DRAWINGS_ATM_CURRENT_divide_CNT_DRAWINGS_ATM_CURRENT
AMT_DRAWINGS_POS_CURRENT_divide_CNT_DRAWINGS_POS_CURRENT
AMT_DRAWINGS_OTHER_CURRENT_divide_CNT_DRAWINGS_OTHER_CURRENT
AMT_DRAWINGS_CURRENT_divide_CNT_DRAWINGS_CURRENT
score_

# Credit: Months Balance max min and sum

In [32]:
_df = credit.groupby('SK_ID_CURR').agg({'MONTHS_BALANCE':['sum','min','max']})
columns = []
for pre in _df.columns.levels[0]:
    for middle in _df.columns.levels[1]:
        columns.append('%s_%s_%s' %('credit_card',pre,middle))
_df.columns = columns
train = train.merge(_df.reset_index(), on = 'SK_ID_CURR', how='left')
test = test.merge(_df.reset_index(), on = 'SK_ID_CURR', how='left')

# Save credit

In [33]:

print(train.shape, test.shape, train.columns.values)
# PATH = '/home/kai/data/kaggle/homecredit/'
# train.to_pickle(PATH + 'train_with_credit.pkl')
# test.to_pickle(PATH + 'test_with_credit.pkl')

(307511, 515) (48744, 514) ['SK_ID_CURR' 'TARGET' 'NAME_CONTRACT_TYPE' 'CODE_GENDER' 'FLAG_OWN_CAR'
 'FLAG_OWN_REALTY' 'CNT_CHILDREN' 'AMT_INCOME_TOTAL' 'AMT_CREDIT'
 'AMT_ANNUITY' 'AMT_GOODS_PRICE' 'NAME_TYPE_SUITE' 'NAME_INCOME_TYPE'
 'NAME_EDUCATION_TYPE' 'NAME_FAMILY_STATUS' 'NAME_HOUSING_TYPE'
 'REGION_POPULATION_RELATIVE' 'DAYS_BIRTH' 'DAYS_EMPLOYED'
 'DAYS_REGISTRATION' 'DAYS_ID_PUBLISH' 'OWN_CAR_AGE' 'FLAG_MOBIL'
 'FLAG_EMP_PHONE' 'FLAG_WORK_PHONE' 'FLAG_CONT_MOBILE' 'FLAG_PHONE'
 'FLAG_EMAIL' 'OCCUPATION_TYPE' 'CNT_FAM_MEMBERS' 'REGION_RATING_CLIENT'
 'REGION_RATING_CLIENT_W_CITY' 'WEEKDAY_APPR_PROCESS_START'
 'HOUR_APPR_PROCESS_START' 'REG_REGION_NOT_LIVE_REGION'
 'REG_REGION_NOT_WORK_REGION' 'LIVE_REGION_NOT_WORK_REGION'
 'REG_CITY_NOT_LIVE_CITY' 'REG_CITY_NOT_WORK_CITY'
 'LIVE_CITY_NOT_WORK_CITY' 'ORGANIZATION_TYPE' 'EXT_SOURCE_1'
 'EXT_SOURCE_2' 'EXT_SOURCE_3' 'APARTMENTS_AVG' 'BASEMENTAREA_AVG'
 'YEARS_BEGINEXPLUATATION_AVG' 'YEARS_BUILD_AVG' 'COMMONAREA_AVG'
 'ELEVATORS_

# POS_CASH balance

In [34]:
# PATH = '/home/kai/data/kaggle/homecredit/'
# train = pd.read_csv(PATH + 'application_train.csv')
# print('train')
# test = pd.read_csv(PATH + 'application_test.csv')
# print('test')
pos = POS_CASH_balance.copy()

In [35]:
pos[substraction_name('CNT_INSTALMENT','CNT_INSTALMENT_FUTURE')] = \
substraction(pos, 'CNT_INSTALMENT','CNT_INSTALMENT_FUTURE')

pos[ratio_name('CNT_INSTALMENT_FUTURE','CNT_INSTALMENT')]\
= ratio(pos,'CNT_INSTALMENT_FUTURE','CNT_INSTALMENT')

pos['score_DPD'] = (pos['SK_DPD']**3 + 5* pos['SK_DPD_DEF']**3 + 1) * (1/(1 + abs(pos['MONTHS_BALANCE']) ))

pos.columns.values

array(['SK_ID_PREV', 'SK_ID_CURR', 'MONTHS_BALANCE', 'CNT_INSTALMENT',
       'CNT_INSTALMENT_FUTURE', 'NAME_CONTRACT_STATUS', 'SK_DPD',
       'SK_DPD_DEF', 'CNT_INSTALMENT_minus_CNT_INSTALMENT_FUTURE',
       'CNT_INSTALMENT_FUTURE_divide_CNT_INSTALMENT', 'score_DPD'], dtype=object)

In [36]:
gp_col = 'SK_ID_CURR'
numerical_cols= ['CNT_INSTALMENT','CNT_INSTALMENT_FUTURE','SK_DPD','SK_DPD_DEF',\
                 'CNT_INSTALMENT_minus_CNT_INSTALMENT_FUTURE','CNT_INSTALMENT_FUTURE_divide_CNT_INSTALMENT','score_DPD']

extrafunc_list = [None,None,['positive_count'],['positive_count'],None,None, None]

for i in range(len(numerical_cols)):
    print(numerical_cols[i])
    test = test.merge(numerical_agg(pos, gp_col, numerical_cols[i], 'POS_CASH_balance', extrafunc_list = extrafunc_list[i]), on = 'SK_ID_CURR', how='left')
    train = train.merge(numerical_agg(pos, gp_col, numerical_cols[i], 'POS_CASH_balance', extrafunc_list = extrafunc_list[i]), on = 'SK_ID_CURR', how='left')

CNT_INSTALMENT
CNT_INSTALMENT_FUTURE
SK_DPD
SK_DPD_DEF
CNT_INSTALMENT_minus_CNT_INSTALMENT_FUTURE
CNT_INSTALMENT_FUTURE_divide_CNT_INSTALMENT
score_DPD


# POS CASH categorical and month

In [37]:
categorical_cols = ['NAME_CONTRACT_STATUS']        

for x in categorical_cols:
    print(x)
    train = train.merge(split_categorical_feature(pos, 'SK_ID_CURR', x), on='SK_ID_CURR', how='left')
    test = test.merge(split_categorical_feature(pos, 'SK_ID_CURR', x), on='SK_ID_CURR', how='left')
    gc.collect()
    
_df = POS_CASH_balance.groupby('SK_ID_CURR').agg({'MONTHS_BALANCE':['sum','min','max']})
columns = []

for pre in _df.columns.levels[0]:
    for middle in _df.columns.levels[1]:
        columns.append('%s_%s_%s' %('POS_CASH_balance',pre,middle))
_df.columns = columns
test = test.merge(_df.reset_index(), on = 'SK_ID_CURR', how='left')
train = train.merge(_df.reset_index(), on = 'SK_ID_CURR', how='left')    

NAME_CONTRACT_STATUS


# Save Pos or Total

In [38]:
print(train.shape, test.shape, train.columns.values)
# PATH = '/home/kai/data/kaggle/homecredit/'
# train.to_pickle(PATH + 'train_with_pos.pkl')
# test.to_pickle(PATH + 'test_with_pos.pkl')

PATH = '/home/kai/data/kaggle/homecredit/'
train.to_pickle(PATH + 'train_total.pkl')
test.to_pickle(PATH + 'test_total.pkl')

(307511, 557) (48744, 556) ['SK_ID_CURR' 'TARGET' 'NAME_CONTRACT_TYPE' 'CODE_GENDER' 'FLAG_OWN_CAR'
 'FLAG_OWN_REALTY' 'CNT_CHILDREN' 'AMT_INCOME_TOTAL' 'AMT_CREDIT'
 'AMT_ANNUITY' 'AMT_GOODS_PRICE' 'NAME_TYPE_SUITE' 'NAME_INCOME_TYPE'
 'NAME_EDUCATION_TYPE' 'NAME_FAMILY_STATUS' 'NAME_HOUSING_TYPE'
 'REGION_POPULATION_RELATIVE' 'DAYS_BIRTH' 'DAYS_EMPLOYED'
 'DAYS_REGISTRATION' 'DAYS_ID_PUBLISH' 'OWN_CAR_AGE' 'FLAG_MOBIL'
 'FLAG_EMP_PHONE' 'FLAG_WORK_PHONE' 'FLAG_CONT_MOBILE' 'FLAG_PHONE'
 'FLAG_EMAIL' 'OCCUPATION_TYPE' 'CNT_FAM_MEMBERS' 'REGION_RATING_CLIENT'
 'REGION_RATING_CLIENT_W_CITY' 'WEEKDAY_APPR_PROCESS_START'
 'HOUR_APPR_PROCESS_START' 'REG_REGION_NOT_LIVE_REGION'
 'REG_REGION_NOT_WORK_REGION' 'LIVE_REGION_NOT_WORK_REGION'
 'REG_CITY_NOT_LIVE_CITY' 'REG_CITY_NOT_WORK_CITY'
 'LIVE_CITY_NOT_WORK_CITY' 'ORGANIZATION_TYPE' 'EXT_SOURCE_1'
 'EXT_SOURCE_2' 'EXT_SOURCE_3' 'APARTMENTS_AVG' 'BASEMENTAREA_AVG'
 'YEARS_BEGINEXPLUATATION_AVG' 'YEARS_BUILD_AVG' 'COMMONAREA_AVG'
 'ELEVATORS_